In [2]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'blue'

In [3]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizVert_wavelet_full_pastis.ipynb


In [4]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [5]:
from scipy import spatial
from sklearn.decomposition import PCA

In [6]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [7]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [8]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [9]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [10]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,19851.58,-0.10,0.21,-0.00,0.00,-0.01,0.0
3,-0.10,5765.37,-0.03,0.03,0.00,0.01,-0.0
4,0.21,-0.03,1361.07,0.02,-0.00,0.00,-0.0
5,-0.00,0.03,0.02,249.95,-0.00,-0.00,0.0
6,0.00,0.00,-0.00,-0.00,35.82,0.00,0.0
7,-0.01,0.01,0.00,-0.00,0.00,4.13,-0.0
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.3


In [11]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,-0.00001,0.00004,-0.00000,0.00001,-0.00003,0.00000
3,-0.00001,1.00000,-0.00001,0.00003,0.00000,0.00005,-0.00002
4,0.00004,-0.00001,1.00000,0.00003,-0.00001,0.00002,-0.00000
5,-0.00000,0.00003,0.00003,1.00000,-0.00001,-0.00001,0.00001
6,0.00001,0.00000,-0.00001,-0.00001,1.00000,0.00002,0.00003
7,-0.00003,0.00005,0.00002,-0.00001,0.00002,1.00000,-0.00001
8,0.00000,-0.00002,-0.00000,0.00001,0.00003,-0.00001,1.00000


In [12]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.33820002539752225

In [13]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.64488227e+07 4.61633411e+06 2.46930800e+05 7.80985600e+03
 1.42712026e+02 1.42597197e+00 1.81120908e-32]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998550,-0.052598,-0.011247,-0.002057,-0.000294,-0.000035,-0.000002
1,0.051947,0.997334,-0.050488,-0.008665,-0.001240,-0.000142,-0.000010
2,0.013734,0.049358,0.997517,-0.047862,-0.006579,-0.000757,-0.000054
3,0.003139,0.010820,0.046960,0.997616,-0.048985,-0.005500,-0.000392
4,0.000598,0.002061,0.008734,0.048199,0.997074,-0.058509,-0.004091
5,0.000104,0.000355,0.001508,0.008220,0.057725,0.995729,-0.071561
6,0.000015,0.000051,0.000216,0.001177,0.008211,0.071197,0.997428


In [14]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.0014497 , 0.00266562, 0.00248279, 0.00238393, 0.00292632,
       0.00427068, 0.00257224])